# Databases Worksheet

This worksheet takes you through migrating the data in our JSON file format, into a 'real' database - what this means, why anyone does this, and what the pros/cons are of different database solutions.

## Learning Objectives
By the end of this worksheet, you will:
- Understand why databases are needed vs file storage
- Know the difference between SQL and NoSQL databases  
- Set up PostgreSQL and MongoDB locally
- Migrate JSON data into both database types
- Write queries in both SQL and MongoDB
- Compare performance between different storage methods

## Table of Contents

1. What is a database, and why use one at all?
2. Types of databases
3. Migrating to SQL
4. Migrating to MongoDB
5. Benchmark Testing

## 1. What is a database, and why use one at all?

A database is a server that acts as a common place to store data. Ideally, databases are:

- easy to access over a network,
- scaleable with entries in the database,
- able to handle requests concurrently, i.e. at the same time,
- predictable

At the moment, this project uses your machine's file system as a database store; when you run the python server and open localhost:8000, the server loads `data.jsonl` into the browser. This is fine for a local project, but there are some inherent limitations, like:

- if you change the data in `data.jsonl`, you need to restart the server to see it reflected in the client (the browser)
- the webpage will perform slower and slower the bigger the dataset is, because each time you load the website the whole list needs to be rendered on the client,
- Even if you've implemented some pagination to prevent the previous point, you have a bigger issue of not being able to deploy your app on any other machine. The website currently needs your laptop to show the 'right' data. If you change that data, it only changes for you locally on your machine! So this solution is not scaleable.
- You can write code functions to filter your data in `data.jsonl`, but there's no real way to 'query' it. As you probably noticed during the filtering task of your assignment, if you want to find red and blue dresses, you need to write a whole new python function for that to work - or at least modify your existing functions - and then probably loop through your dataset. This is slow and inefficient, especially with larger datasets. Databases provide a query language, by which you can efficiently and quickly retrieve records (entries in the database) based on a flexible criteria.

For these reasons, software products almost always come with non-locally hosted databases that serve their content.

## 2. Types of databases

There are two types of databases; relational, and non-relational.

**Relational** databases have data that is structured into tables. All relational database servers (RDS) use SQL, or Structured Query Language, to allow clients to access entries in the database. SQL isn't _really_ a programming language, but a framework to query data. The 'flavor' of SQL you'll use will differ depending on which RDS you go for. Widely used free open-source options include [MySQL](https://www.mysql.com/), [PostgreSQL](https://www.postgresql.org/) and [MariaDB](https://mariadb.org/). There are commercial options around like Oracle, Microsoft's SQL server and ones associated with any cloud network like AWS or Azure.

**Non-relational** databases have non structured data. These can contain anything from graphs (such as [neo4j](https://neo4j.com/docs/)), to documents (such as [mongodb](https://www.mongodb.com/docs/)), to key-value pairs (like [redis](https://redis.io/docs/latest/)).

Which type you go for, and exactly which server you choose (i.e. MySQL vs Postgresql) will depend on:

- the kind of data you need to surface,
- how you want to query that data, i.e. if you need to store customer details and client details, you might want to use SQL if you think you'll need to ever join them together (i.e. you want to find out which of your customers have done business with which clients),
- personal preference on the specific RDS, for example the type of SQL you write with PostgreSQL is different to what you'd use for MariaDB... while it's largely similar-looking, the exact functions you have access to will differ between servers.

In [ ]:
# Visual comparison of database types
import pandas as pd

# Create a comparison table
comparison_data = {
    'Aspect': [
        'Data Structure',
        'Query Language', 
        'Scalability',
        'Schema',
        'Use Case'
    ],
    'Relational (SQL)': [
        'Tables with rows/columns',
        'SQL',
        'Vertical scaling',
        'Fixed schema',
        'Complex relationships, transactions'
    ],
    'Document (NoSQL)': [
        'Documents (JSON-like)',
        'Native queries',
        'Horizontal scaling', 
        'Flexible schema',
        'Rapid development, unstructured data'
    ]
}

df = pd.DataFrame(comparison_data)
print("Database Type Comparison:")
print("=" * 100)
for _, row in df.iterrows():
    print(f"{row['Aspect']:20} | {row['Relational (SQL)']:35} | {row['Document (NoSQL)']}")
print("=" * 100)

Database Type Comparison:
Data Structure       | Tables with rows/columns            | Documents (JSON-like)
Query Language       | SQL                                 | Native queries
Scalability          | Vertical scaling                    | Horizontal scaling
Schema               | Fixed schema                        | Flexible schema
Use Case             | Complex relationships, transactions | Rapid development, unstructured data


## 3. Using a relational database: PostgreSQL

We are going to migrate the data currently in `data.jsonl` and put it in a relational database first.

Ironically, we're actually going to set up a local version of a PostgreSQL server on your machine - the very problem we're trying to get away from! But the key thing to note here is, it doesn't _have_ to be hosted on your machine - any of the database servers mentioned in this document can be hosted on a remote machine, i.e. a remote server, and you can usually authenticate to it and query it in just the same way as we're going to demonstrate both for this task (3.) and the next (4.). There is not such an option for file system storage; while you _could_ connect to a remote machine via SSH for example, and access its files, querying them is a lot more inefficient.

### 3.1 Setting up a PostgreSQL server locally

#### 3.1.1 Basic installation and setup
We can install postgres on your laptop by following the [official documentation](https://www.postgresql.org/download/macosx/)... A lot of the commands here are also taken from the relevant section of [this guide](https://www.tigerdata.com/learn/how-to-install-postgresql-on-macos).

```bash
brew install postgresql
```

This will take a couple of minutes to run. After this, you can start the server in the background.

```bash
brew services start postgresql
```

... and verify it's running with,

```bash
brew services info postgresql
```

which for me shows something similar to:

```bash
postgresql@14 (homebrew.mxcl.postgresql@14)
Running: ✔
Loaded: ✔
Schedulable: ✘
User: rosesyrett
PID: 75325
```

Every time you log into your machine, it should spin up a local postgres server for you. You can verify the location of the postgres data storage by connecting to the server using

```bash
psql postgres
```

and then typing this command,
```postgres
postgres=# SHOW data_directory;
         data_directory
---------------------------------
 /opt/homebrew/var/postgresql@14
 ```

#### 3.1.2 Moving data into your locally hosted DB

From the previous section, you _could_ in theory make a bunch of 'INSERT' statements to insert a record into a new table that you create in a postgres server terminal.

However, because we have 10,000 records to enter, you might spend a long time doing this! So a more efficient way is to download a python client library to connect to your database instead, and programatically read the contents of `data.jsonl` into the database. We only need to do this once - we can rest assured that the data location of the local postgres server isn't going to get wiped. The most popular client library for python is [psycopg](https://www.psycopg.org/docs/) which we will use now.

##### 3.1.2.1 Add psycopg into requirements.txt

The easiest way to install any new package into your project, is find it in pypi, the online python package repository. [Here is the page for psycopg2](https://pypi.org/project/psycopg2/).

From here, navigate to the 'releases' and find the latest stable release. We can see [from the prerequisites](https://www.psycopg.org/docs/install.html#prerequisites) that psycopg2 supports postgresql versions 7.4 to 18, and in the previous section we've spotted that it installed version 14 for me - so we know we're good to go with the latest stable version, which at the time of writing is 2.9.11.

Therefore, just add this line into `requirements.txt`:

```
psycopg2>=2.9.11
```

##### 3.1.2.2 Install requirements.txt into your virtual environment

Active your virtual environment...

```bash
source venv/bin/activate
```

... and install the requirements into it

```bash
pip install -r requirements.txt
```

##### 3.1.2.3 Connect to your db from python

You can connect to a postgres instance by using `psycopg2.connect`, passing the database name, user, password and port. Some of these have default values we don't need to supply, such as the port (which is always 5432 when running locally), but in general `brew services start postgresql` will spin up a postgres database with:

- database name of "postgres" - you can see this as the text before where you type commands in the server terminal, see the code block at the bottom of section 3.1.1
- username which is the same as your current user, but you can also find this inside the server terminal by typing the command `\du`.
- an empty password.

Therefore, for me to connect to my instance I need to run the following...

In [7]:
import psycopg

# Test connection to verify setup
with psycopg.connect(dbname="postgres", user="rosesyrett", password="") as conn:
    print("Connected to PostgreSQL successfully!")
    print(f"Server version: {conn.info.server_version}")

Connected to PostgreSQL successfully!
Server version: 140020


##### 3.1.2.4 Run a batch of insert statements to copy data to the DB

To insert data into our database, we're going to need to create a database table. That means we need to know all the fields we want to insert, as well as their datatypes.

PostgreSQL supports several different datatypes, which are [documented here](https://www.postgresql.org/docs/current/datatype.html#DATATYPE-TABLE).

A single product contains the following fields:

```json
{
    "product_id": 1055705468,
    "color": "blue",
    "gender": "M",
    "product_type": "clothing",
    "category": "jackets",
    "subcategory": "waistcoats",
    "designer": "dolce-gabbana",
    "retailer": "thecorner",
    "on_sale": False,
    "regular_price": 1198.00,
    "discount_price": 1198.00,
    "short_description": "...",
    "long_description": "...",
    "image_url": "...",
    "combined_score": 2.74026,
    "popularity_score": 0.85,
    "conversion_score": 1.51859
}
```

So to convert them into something PostgreSQL can understand we should think about the datatypes for each field.

An `integer` type has the description, 'signed eight-byte integer'. What does this mean, and can we store our product_id with this type?

###### Aside: bits and bytes

1 byte contains 8 bits, and computers only have so much memory they allocate for numbers. Floats are similar; there's only so many decimal places you can store a float as. You might have come across this in python, especially numpy, with types like `np.int32` or `np.int64` - this means 32 bit and 64 bit integers respectively.

An unsigned int means a positive number. A signed int means the sign (i.e. + or -) is stored as part of the memory representation of that number, so it can be positive or negative. Either way, there are maximal values that these numbers can physically be in memory, because of how many bits they take up.

In general, to calculate how big your base 10 number (e.g. 1055705468) can get based on the number of bits you can store it in, use this formula...

$
2^{N} - 1
$

... where $N$ is the number of bits. So an unsigned `np.int32` can store any number from 0 to $2^{32} - 1 = 4294967295$, and an unsigned integer can store half of this (half of it with a minus sign, half with a plus).

Therefore, is the datatype 'integer' enough for our use case? Each integer is 4 bytes, and its signed, so that means we can have 

$
\frac{2^{8*4} - 1}{2} = 2147483648
$

Let's find programatically if we can use 4 byte or 8 byte integers...

In [6]:
from filter import load_products

products = load_products()

# check all product_ids are unique...

seen_ids = {product["product_id"] for product in products}

assert len(seen_ids) == len(products), "product IDs are not unique!"

print("Is 8 byte integer enough space for the product IDs? ", max(seen_ids) < 2**(8*4) -1 )
print("Is 4 byte integer enough space for the product IDs? ", max(seen_ids) < 2**(4*4) -1 )

Is 8 byte integer enough space for the product IDs?  True
Is 4 byte integer enough space for the product IDs?  False


In [8]:
import psycopg
from filter import load_products

products = load_products()

TABLE_NAME = "products"

# Connect to an existing database
with psycopg.connect(dbname="postgres", user="rosesyrett", password="") as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # First, check if the 'products' table already exists! This will be true if you re-run this cell...
        cur.execute("select exists(select * from information_schema.tables where table_name=%s)", (TABLE_NAME,))
        table_exists = cur.fetchone()[0]

        if not table_exists:
            print("table does not exist! Creating table: ", TABLE_NAME)
            # Execute a command: this creates a new 'products' table if it doesn't already exist!
            cur.execute("""
                CREATE TABLE {} (
                    product_id bigint PRIMARY KEY,
                    color text,
                    gender text,
                    product_type text,
                    category text,
                    subcategory text,
                    designer text,
                    retailer text,
                    on_sale boolean,
                    regular_price float,
                    discount_price float,
                    short_description text,
                    long_description text,
                    image_url text,
                    combined_score float,
                    popularity_score float,
                    conversion_score float
                )
                """.format(TABLE_NAME))
    
            # Pass data to fill a query placeholders and let Psycopg perform
            # the correct conversion (no SQL injections!)
            cur.executemany(
                """INSERT INTO {} (
                    product_id,
                    color,
                    gender,
                    product_type,
                    category,
                    subcategory,
                    designer, 
                    retailer, 
                    on_sale, 
                    regular_price, 
                    discount_price, 
                    short_description, 
                    long_description, 
                    image_url,
                    combined_score,
                    popularity_score,
                    conversion_score
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""".format(TABLE_NAME),
                [tuple(product.values()) for product in products]
            )
            print(f"Inserted {len(products)} products into the database!")
        else:
            print("Table already exists, skipping creation and data insertion.")

        # Query the database and obtain data as Python objects.
        cur.execute(f"SELECT * FROM {TABLE_NAME} LIMIT 1;")
        sample_record = cur.fetchone()
        print(f"Sample record from the {TABLE_NAME} table: {sample_record}")

        # Get total count
        cur.execute(f"SELECT COUNT(*) FROM {TABLE_NAME};")
        total_count = cur.fetchone()[0]
        print(f"Total records in database: {total_count}")

        # Make the changes to the database persistent
        conn.commit()

Table already exists, skipping creation and data insertion.
Sample record from the products table: (1179406705, 'blue', 'M', 'clothing', 'jackets', 'formal jackets', 'dolce-gabbana', 'seymayka', True, 3733.0, 1450.7, 'Navy Blue Double Breasted Suit Coat Blazer', 'DOLCE & GABBANAGorgeous brand new, 100% Authentic DOLCE & GABBANA suit has two button and three pockets. Model: Formal blazer suitStyle: Double breasted two buttonFitting: RegularColor: Navy BlueMaterial: 88% Flax 11% Virgin Wool 1% ElastaneLining: 70% Cotton 30% SilkThree outside pockets and two inside pocketsOne open vent in the backLogo detailsMade in ItalyVery exclusive and high craftsmanship.', 'https://cdna.lystit.com/300/379/tr/photos/seymayka/c725f533/dolce-gabbana--Navy-Blue-Double-Breasted-Suit-Coat-Blazer.jpeg', 3.18095, 3.45619, 2.38633)
Total records in database: 10000


### 3.2 Querying the DB

There are many ways to query the database. You could go directly through a postgresql terminal, like this...

```sql
postgres=# SELECT count(*) FROM products WHERE color = 'black';
 count
-------
  2998
(1 row)
```

You could also use the python library similarly to what we did above...

In [9]:
import psycopg

with psycopg.connect(dbname="postgres", user="rosesyrett", password="") as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Query the database and obtain data as Python objects.
        cur.execute("SELECT * FROM products WHERE color = 'black';")
        black_products = cur.fetchall()

print("Total number of black products: ", len(black_products))
if black_products:
    print("First black product looks like: ", black_products[0])
else:
    print("No black products found!")

Total number of black products:  2998
First black product looks like:  (1151539922, 'black', 'F', 'shoes', 'flats', 'flat sandals', 'tj-maxx', 'tjmaxx', True, 99.99, 67.0, 'Clementine Slingback Flats For Leather', 'Lightly Padded Footbed, Low Block Heel, Glitter Finish, Textured Design, Leather Lining, Jewel Embellished Hardware Detail, Size Chart Conversion May Vary Slightly By Country, Goring Detail For Stretch, Pointy Toe, Man Made Sole, Slingback Closure, Textile Upper, Imported, Made In Spain | Clementine Slingback Flats For Women, Leather', 'https://cdna.lystit.com/300/379/tr/photos/tjmaxx/55d968b7/tj-maxx-Sparkle-Black-Clementine-Slingback-Flats-For-Leather.jpeg', 1.4149, 1.8213, 4.61333)


Hopefully you can see how we could replace some of the filtering on our website with calls to the database!

**As an exercise**, make a new branch of your repository called `postgres` and replace all the places in your code where you would normally read from `data.jsonl` with something that will fetch products, similarly to what we've done in the code snippet above.

## 4. Using a non-relational database: MongoDB

You've just played around with a relational database, which are great when you have multiple tables that relate to each other. In our case, we have a simple database with no relations yet - so, we can decide to use a non-relational database instead, like mongodb.

For this section, it is recommended you check out your main branch and make a new branch, called 'mongodb' or similar, to do this task in.

### 4.1 Setting up MongoDB locally

#### 4.1.1 Basic installation and setup

Mongodb publishes some good [documentation](https://www.mongodb.com/docs/manual/administration/install-community/?operating-system=macos&macos-installation-method=homebrew) on how to do this, but the commands you need to use are summarised here:

```bash
$ xcode-select --install
$ brew tap mongodb/brew
$ brew update
$ brew install mongodb-community@8.2
```

... Then, to start mongodb locally use this command...
```bash
$ brew services start mongodb-community@8.2
```

... now, if you list your services you'll see both mongodb and postgres running locally:
```bash
$ brew services list
Name              Status  User       File
mongodb-community started rosesyrett ~/Library/LaunchAgents/homebrew.mxcl.mongodb-community.plist
postgresql@14     started rosesyrett ~/Library/LaunchAgents/homebrew.mxcl.postgresql@14.plist
```

Finally, just like with postgres, mongodb comes equipped with a handy CLI tool that lets us connect to instances of these databases. For mongo, we can use this command:

```bash
$ mongosh
Current Mongosh Log ID:	699330fdf60262079406e71c
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.7.0
Using MongoDB:		8.2.5
Using Mongosh:		2.7.0

For mongosh info see: https://www.mongodb.com/docs/mongodb-shell/
test> 
```

#### 4.1.2 Moving data into your locally hosted DB

Mongodb provides a python client, to connect to mongo databases, which is [documented here](http://mongodb.com/docs/languages/python/pymongo-driver/current/connect/mongoclient/).

##### 4.1.2.1 Add pymongo into requirements.txt

The mongo documentation references pymongo, which is documented in pypi [here](https://pypi.org/project/pymongo/).

The latest stable release at the time of writing is 4.16.0, so we can just add this line into `requirements.txt`:

```
pymongo>=4.16.0
```

##### 4.1.2.2 Install requirements.txt into your virtual environment

Active your virtual environment...

```bash
source venv/bin/activate
```

... and install the requirements into it

```bash
pip install -r requirements.txt
```

##### 4.1.2.3 Connect to your db from python

To connect to the db, we will need the URI of our mongodb. Notice in the previous section, when you ran `mongosh` it gave you this information - which means you should just be able to connect with this! By default, your local database does not have a username and password, but for a 'real' database (probably hosted on another server, i.e. not just your local machine) you would need to provide these details when creating the client.

In [ ]:
from pymongo import MongoClient

uri = "mongodb://localhost:27017/" # note: The IP address 127.0.0.1 is the same as localhost! It just means it runs on your machine only.
client = MongoClient(uri)

##### 4.1.2.4 Copy data to the DB

MongoDB stores JSON blobs in collections, and collections in databases. Right now, if you go to a mongo shell, you can see you have no collections and a few small databases:

```mongodb
test> show collections

test> show databases
admin   40.00 KiB
config  60.00 KiB
local   40.00 KiB
```

A collection is similar to a database table, like the 'products' table we made in the previous section. We can go ahead and create a new database and a new collection for our products to live...

In [10]:
from pymongo import MongoClient
from filter import load_products

products = load_products()

uri = "mongodb://localhost:27017/" # note: The IP address 127.0.0.1 is the same as localhost! It just means it runs on your machine only.
client = MongoClient(uri)

# create a database
database = client["test_database"]

# create a collection inside that database to store our products
collection = database["products"]

if collection.count_documents({}) == 0:
    result = collection.insert_many(products)

count = collection.count_documents({})
print(collection, "has", count, "documents!")

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database'), 'products') has 10000 documents!


### 4.2 Querying the DB

Just like with postgres, there are multiple ways to query the database. If you wanted to use the mongo shell, you could simply enter it like so...
```bash
$ mongosh
```

... and then go into your new database, and filter objects in the collection
```mongosh
>test use test_database
switched to db test_database
test_database> show collections
products
test_database> db.products.find({})
[
    ...
    {
        _id: ObjectId('699335ae4db0be355d8e5cd1'),
        product_id: 1087619417,
        color: 'gray',
        gender: 'M',
        product_type: 'clothing',
        category: 'coats',
        subcategory: 'trench coats',
        designer: 'burberry',
        retailer: 'senser',
        on_sale: true,
        regular_price: 1802.5,
        discount_price: 1687.5,
        short_description: 'Double-Breasted Long Trench Coat',
        long_description: 'Double-breasted long trench coat, long sleeves, lapel, waist tie details.',
        image_url: 'https://cdna.lystit.com/300/379/tr/photos/senser/d2643d6e/burberry--Double-Breasted-Long-Trench-Coat.jpeg',
        combined_score: 2.83343,
        popularity_score: 2.1646,
        conversion_score: 1.55012
    }
]
Type "it" for more
test_database> db.products.find({ color: "black"})
...
test_database> db.products.countDocuments({ color: "black"})
2998
```

You could also use the python library similarly to what we did above...

In [11]:
black_products = collection.find({ "color": "black" })

print(black_products)

for i in range(3):
    print(next(black_products))

{'_id': ObjectId('699335ae4db0be355d8e5cc1'), 'product_id': 1151539922, 'color': 'black', 'gender': 'F', 'product_type': 'shoes', 'category': 'flats', 'subcategory': 'flat sandals', 'designer': 'tj-maxx', 'retailer': 'tjmaxx', 'on_sale': True, 'regular_price': 99.99, 'discount_price': 67.0, 'short_description': 'Clementine Slingback Flats For Leather', 'long_description': 'Lightly Padded Footbed, Low Block Heel, Glitter Finish, Textured Design, Leather Lining, Jewel Embellished Hardware Detail, Size Chart Conversion May Vary Slightly By Country, Goring Detail For Stretch, Pointy Toe, Man Made Sole, Slingback Closure, Textile Upper, Imported, Made In Spain | Clementine Slingback Flats For Women, Leather', 'image_url': 'https://cdna.lystit.com/300/379/tr/photos/tjmaxx/55d968b7/tj-maxx-Sparkle-Black-Clementine-Slingback-Flats-For-Leather.jpeg', 'combined_score': 1.4149, 'popularity_score': 1.8213, 'conversion_score': 4.61333}
{'_id': ObjectId('699335ae4db0be355d8e5cc2'), 'product_id': 116

Notice how the return of `collection.find` isn't just all the objects, but an instance of `pymongo.synchronous.cursor.Cursor` instead... so to do something with all the black products, you'd have to iterate through them like so:

In [12]:
for product in black_products:
    ... # do something here!

**As an exercise**, in your `mongodb` branch that you created at the start of this section, replace all the places in your code where you would normally read from `data.jsonl` with something that will fetch products, similarly to what we've done in the code snippet above.

## 5. Benchmark Testing

You have now migrated your data from `data.jsonl` into two databases, and done this in two separate branches. Now, it's time to do some testing, to see how much longer or slower the different approaches of data storage and retrieval that we have are!

Just by navigating on the website on the main branch, i.e. not having migrated any of the data to any database, we make 'database' calls whenever we need to access the data in `data.jsonl`, especially when we apply filters.

So... let's do some benchmark testing!

**As an exercise**, switch between the branches `mongodb` and `postgres` and check how long you think it takes to apply a filter on the website. You should be able to do this using google chrome's inspector tools, but you can also add some `time.time()` statements in the codebase and some `print` statements, where you should see your server print out how long it takes.

We want to answer the following questions:

- with the dataset of products that we have, which option is the fastest? (i.e. file system loading vs postgres vs mongodb)
- How consistent is this? Can you write a test file like `test_timings.py` in both of your branches which will tell you how long a query takes over N amount of iterations?